# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [4]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [5]:
env = gym.make('BreakoutDeterministic-v4')
env.render()

True

In [6]:
number_lives = find_max_lifes(env)
state_size = env.observation_space.shape
action_size = 3
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [7]:
agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0


### Main Training Loop

In [ ]:
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        if render_breakout:
            env.render()

        # Select and perform an action
        action = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action + 1)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        r = np.clip(reward, -1, 1)

        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
        # Start training after random sample generation
        if(frame >= train_frame):
            agent.train_policy_net(frame)
            # Update the target network
            if(frame % Update_target_network_frequency)== 0:
                agent.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/breakout_dqn.png")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 10:
                torch.save(agent.model, "./save_model/breakout_dqn")
                sys.exit()

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


episode: 0   score: 0.0   memory length: 135   epsilon: 1.0    steps: 135     evaluation reward: 0.0
episode: 1   score: 2.0   memory length: 336   epsilon: 1.0    steps: 201     evaluation reward: 1.0
episode: 2   score: 3.0   memory length: 590   epsilon: 1.0    steps: 254     evaluation reward: 1.6666666666666667
episode: 3   score: 1.0   memory length: 770   epsilon: 1.0    steps: 180     evaluation reward: 1.5
episode: 4   score: 0.0   memory length: 909   epsilon: 1.0    steps: 139     evaluation reward: 1.2
episode: 5   score: 1.0   memory length: 1082   epsilon: 1.0    steps: 173     evaluation reward: 1.1666666666666667
episode: 6   score: 0.0   memory length: 1222   epsilon: 1.0    steps: 140     evaluation reward: 1.0
episode: 7   score: 1.0   memory length: 1378   epsilon: 1.0    steps: 156     evaluation reward: 1.0
episode: 8   score: 0.0   memory length: 1507   epsilon: 1.0    steps: 129     evaluation reward: 0.8888888888888888
episode: 9   score: 1.0   memory length: 1

episode: 72   score: 0.0   memory length: 13346   epsilon: 1.0    steps: 135     evaluation reward: 1.3424657534246576
episode: 73   score: 2.0   memory length: 13554   epsilon: 1.0    steps: 208     evaluation reward: 1.3513513513513513
episode: 74   score: 1.0   memory length: 13727   epsilon: 1.0    steps: 173     evaluation reward: 1.3466666666666667
episode: 75   score: 1.0   memory length: 13880   epsilon: 1.0    steps: 153     evaluation reward: 1.3421052631578947
episode: 76   score: 0.0   memory length: 14005   epsilon: 1.0    steps: 125     evaluation reward: 1.3246753246753247
episode: 77   score: 2.0   memory length: 14212   epsilon: 1.0    steps: 207     evaluation reward: 1.3333333333333333
episode: 78   score: 1.0   memory length: 14382   epsilon: 1.0    steps: 170     evaluation reward: 1.3291139240506329
episode: 79   score: 0.0   memory length: 14511   epsilon: 1.0    steps: 129     evaluation reward: 1.3125
episode: 80   score: 2.0   memory length: 14714   epsilon: 1

episode: 147   score: 1.0   memory length: 26315   epsilon: 1.0    steps: 156     evaluation reward: 1.24
episode: 148   score: 2.0   memory length: 26518   epsilon: 1.0    steps: 203     evaluation reward: 1.22
episode: 149   score: 1.0   memory length: 26695   epsilon: 1.0    steps: 177     evaluation reward: 1.19
episode: 150   score: 0.0   memory length: 26825   epsilon: 1.0    steps: 130     evaluation reward: 1.18
episode: 151   score: 1.0   memory length: 26985   epsilon: 1.0    steps: 160     evaluation reward: 1.18
episode: 152   score: 1.0   memory length: 27139   epsilon: 1.0    steps: 154     evaluation reward: 1.19
episode: 153   score: 1.0   memory length: 27322   epsilon: 1.0    steps: 183     evaluation reward: 1.18
episode: 154   score: 1.0   memory length: 27499   epsilon: 1.0    steps: 177     evaluation reward: 1.16
episode: 155   score: 2.0   memory length: 27686   epsilon: 1.0    steps: 187     evaluation reward: 1.17
episode: 156   score: 1.0   memory length: 278

episode: 225   score: 2.0   memory length: 39250   epsilon: 1.0    steps: 185     evaluation reward: 0.87
episode: 226   score: 0.0   memory length: 39376   epsilon: 1.0    steps: 126     evaluation reward: 0.87
episode: 227   score: 1.0   memory length: 39558   epsilon: 1.0    steps: 182     evaluation reward: 0.88
episode: 228   score: 0.0   memory length: 39680   epsilon: 1.0    steps: 122     evaluation reward: 0.86
episode: 229   score: 0.0   memory length: 39822   epsilon: 1.0    steps: 142     evaluation reward: 0.86
episode: 230   score: 1.0   memory length: 39995   epsilon: 1.0    steps: 173     evaluation reward: 0.86
episode: 231   score: 1.0   memory length: 40152   epsilon: 1.0    steps: 157     evaluation reward: 0.87
episode: 232   score: 3.0   memory length: 40382   epsilon: 1.0    steps: 230     evaluation reward: 0.89
episode: 233   score: 2.0   memory length: 40570   epsilon: 1.0    steps: 188     evaluation reward: 0.9
episode: 234   score: 3.0   memory length: 4081

...

episode: 1552   score: 1.0   memory length: 302744   epsilon: 0.7992824500043574    steps: 175     evaluation reward: 3.2
episode: 1553   score: 2.0   memory length: 302930   epsilon: 0.7990983100043614    steps: 186     evaluation reward: 3.21
episode: 1554   score: 2.0   memory length: 303137   epsilon: 0.7988933800043658    steps: 207     evaluation reward: 3.2
episode: 1555   score: 2.0   memory length: 303364   epsilon: 0.7986686500043707    steps: 227     evaluation reward: 3.19
episode: 1556   score: 5.0   memory length: 303664   epsilon: 0.7983716500043772    steps: 300     evaluation reward: 3.22
episode: 1557   score: 5.0   memory length: 303978   epsilon: 0.7980607900043839    steps: 314     evaluation reward: 3.24
episode: 1558   score: 3.0   memory length: 304194   epsilon: 0.7978469500043885    steps: 216     evaluation reward: 3.23
episode: 1559   score: 3.0   memory length: 304415   epsilon: 0.7976281600043933    steps: 221     evaluation reward: 3.2
episode: 1560   sco

episode: 1619   score: 7.0   memory length: 319858   epsilon: 0.7823395900047252    steps: 420     evaluation reward: 3.28
episode: 1620   score: 2.0   memory length: 320061   epsilon: 0.7821386200047296    steps: 203     evaluation reward: 3.26
episode: 1621   score: 3.0   memory length: 320287   epsilon: 0.7819148800047344    steps: 226     evaluation reward: 3.24
episode: 1622   score: 1.0   memory length: 320440   epsilon: 0.7817634100047377    steps: 153     evaluation reward: 3.22
episode: 1623   score: 6.0   memory length: 320748   epsilon: 0.7814584900047443    steps: 308     evaluation reward: 3.21
episode: 1624   score: 5.0   memory length: 321039   epsilon: 0.7811704000047506    steps: 291     evaluation reward: 3.23
episode: 1625   score: 5.0   memory length: 321351   epsilon: 0.7808615200047573    steps: 312     evaluation reward: 3.25
episode: 1626   score: 1.0   memory length: 321525   epsilon: 0.780689260004761    steps: 174     evaluation reward: 3.24
episode: 1627   s

episode: 1686   score: 4.0   memory length: 335459   epsilon: 0.7668946000050605    steps: 299     evaluation reward: 3.13
episode: 1687   score: 1.0   memory length: 335633   epsilon: 0.7667223400050642    steps: 174     evaluation reward: 3.08
episode: 1688   score: 3.0   memory length: 335896   epsilon: 0.7664619700050699    steps: 263     evaluation reward: 3.06
episode: 1689   score: 3.0   memory length: 336142   epsilon: 0.7662184300050752    steps: 246     evaluation reward: 3.05
episode: 1690   score: 0.0   memory length: 336270   epsilon: 0.7660917100050779    steps: 128     evaluation reward: 2.96
episode: 1691   score: 4.0   memory length: 336545   epsilon: 0.7658194600050838    steps: 275     evaluation reward: 2.97
episode: 1692   score: 6.0   memory length: 336890   epsilon: 0.7654779100050912    steps: 345     evaluation reward: 2.98
episode: 1693   score: 5.0   memory length: 337193   epsilon: 0.7651779400050978    steps: 303     evaluation reward: 3.0
episode: 1694   s

episode: 1753   score: 3.0   memory length: 352315   epsilon: 0.7502071600054228    steps: 238     evaluation reward: 3.12
episode: 1754   score: 2.0   memory length: 352510   epsilon: 0.750014110005427    steps: 195     evaluation reward: 3.14
episode: 1755   score: 4.0   memory length: 352799   epsilon: 0.7497280000054332    steps: 289     evaluation reward: 3.16
episode: 1756   score: 5.0   memory length: 353122   epsilon: 0.7494082300054401    steps: 323     evaluation reward: 3.19
episode: 1757   score: 4.0   memory length: 353391   epsilon: 0.7491419200054459    steps: 269     evaluation reward: 3.22
episode: 1758   score: 3.0   memory length: 353643   epsilon: 0.7488924400054513    steps: 252     evaluation reward: 3.2
episode: 1759   score: 5.0   memory length: 354010   epsilon: 0.7485291100054592    steps: 367     evaluation reward: 3.22
episode: 1760   score: 4.0   memory length: 354275   epsilon: 0.7482667600054649    steps: 265     evaluation reward: 3.24
episode: 1761   sc

episode: 1820   score: 9.0   memory length: 369899   epsilon: 0.7327990000058007    steps: 333     evaluation reward: 3.51
episode: 1821   score: 6.0   memory length: 370265   epsilon: 0.7324366600058085    steps: 366     evaluation reward: 3.49
episode: 1822   score: 2.0   memory length: 370446   epsilon: 0.7322574700058124    steps: 181     evaluation reward: 3.51
episode: 1823   score: 3.0   memory length: 370672   epsilon: 0.7320337300058173    steps: 226     evaluation reward: 3.51
episode: 1824   score: 4.0   memory length: 370951   epsilon: 0.7317575200058233    steps: 279     evaluation reward: 3.53
episode: 1825   score: 6.0   memory length: 371283   epsilon: 0.7314288400058304    steps: 332     evaluation reward: 3.55
episode: 1826   score: 4.0   memory length: 371544   epsilon: 0.731170450005836    steps: 261     evaluation reward: 3.55
episode: 1827   score: 5.0   memory length: 371860   epsilon: 0.7308576100058428    steps: 316     evaluation reward: 3.58
episode: 1828   s